### This script generates 10 possible trial list .csv files to be used for OL task as part of the battery
##### The following parameters are counterbalanced across the different versions:
- block order across trial lists
- vertical order of colors across blocks and trial lists
- horizontal order of colors across blocks
- token colors??

In [12]:
from __future__ import absolute_import, division
import sys  # to get file system encoding
import numpy as np  # whole numpy lib is available, prepend 'np.'
from numpy import (sin, cos, tan, log, log10, pi, average,
                   sqrt, std, deg2rad, rad2deg, linspace, asarray)
from numpy.random import random, randint, normal, shuffle
import pandas as pd
import itertools
import os  # handy system and path functions
import xlsxwriter
from xlrd import open_workbook

In [13]:
#randomize run order across trial lists
run_order = np.array([[1,2,3,4,2,1,4,3],[1,2,3,4,2,1,4,3],[2,3,4,1,3,4,1,2],[2,3,4,1,3,4,1,2],
                      [4,3,2,1,3,4,1,2],[4,3,2,1,3,4,1,2],[3,1,4,2,1,4,2,3],[3,1,4,2,1,4,2,3],
                      [1,4,3,2,4,1,2,3],[1,4,3,2,4,1,2,3]])
unc_order = np.array([[1,2,1,2,1,2,1,2],[2,1,2,1,2,1,2,1],[1,2,1,2,1,2,1,2],[2,1,2,1,2,1,2,1],
                      [1,2,1,2,1,2,1,2],[2,1,2,1,2,1,2,1],[1,2,1,2,1,2,1,2],[2,1,2,1,2,1,2,1],
                      [1,2,1,2,1,2,1,2],[2,1,2,1,2,1,2,1]])

#randomize vertical order of colors across blocks and trial lists
#1= green (top), red (middle), blue (bottom)
#2= red (top), blue (middle), green (bottom)
#3= blue (top), green (middle), red (bottom)
vert_order = np.array([[2,3,1,1,2,3,3,1],[1,2,3,1,3,2,3,1],[2,3,1,1,2,3,2,1],[1,3,2,3,1,2,2,1],
                       [3,1,2,3,2,1,1,2],[2,3,1,3,1,2,2,1],[2,3,1,3,2,1,3,2],[3,1,2,3,2,1,2,1],
                       [3,1,2,2,1,3,3,1],[1,2,3,1,3,2,2,3]])

#randomize horizontal order of colors across blocks
#1= green (left), red (middle), blue (right)
#2= red (left), blue (middle), green (right)
#3= blue (left), green (middle), red (right)
horiz_order = np.array([[3,1,2,1,2,3,1,3],[2,1,3,3,1,2,1,3],[2,1,3,2,3,1,2,3],[1,2,3,2,1,3,3,2],
                        [1,3,2,1,3,2,3,1],[1,2,3,2,3,1,3,1],[2,3,1,1,2,3,3,2],[3,2,1,1,3,2,1,2],
                        [2,3,1,3,1,2,3,2],[3,1,2,3,1,2,3,1]])

#randomize valuable token: 1=as it is now, 2=G->R, R->B, B->G
valtok_order = np.array([1,1,1,2,2,1,2,2,1,2])

In [15]:
listv = [1,2,3,4,5,6,7,8,9,10]
for l in listv:
    print(l)
    ro = run_order[l-1]
    uc = unc_order[l-1]
    vo = vert_order[l-1]
    ho = horiz_order[l-1]
    to = valtok_order[l-1]
    tr_list_all = {'runNb':[], 'runID':[], 'trialNb':[], 'trType':[], 'goalToken':[], 'uncertainty':[], 
               'unavAct':[], 'corrAct':[], 'bestAct':[], 'vertOrd':[], 'horizOrd':[]}
    tp = 0
    run_nb = 1
    for run in ro:
        run_fname = 'run%i.xlsx' %(run)
        wb = open_workbook(run_fname)
        s = wb.sheet_by_index(0)
        for col in range(s.ncols-3):
            key_name = str(s.cell(0,col).value)
            if key_name != 'uncertainty':
                key_vals = [int(num) for num in s.col_values(col,start_rowx=1,end_rowx=s.nrows)]
                tr_list_all[key_name] = tr_list_all[key_name] + key_vals
            
        tr_list_all['runNb'] = tr_list_all['runNb'] + [run_nb for i in range(s.nrows-1)]
        tr_list_all['uncertainty'] = tr_list_all['uncertainty'] + [uc[run_nb-1] for i in range(s.nrows-1)]
        tr_list_all['vertOrd'] = tr_list_all['vertOrd'] + [vo[run_nb-1] for i in range(s.nrows-1)]
        tr_list_all['horizOrd'] = tr_list_all['horizOrd'] + [ho[run_nb-1] for i in range(s.nrows-1)]
        
        TT = [int(num) for num in s.col_values(2,start_rowx=1,end_rowx=s.nrows)] #vector of trial type (2=play)
        UA = [int(num) for num in s.col_values(5,start_rowx=1,end_rowx=s.nrows)] #vector of unavailable actions
        CA = [int(num) for num in s.col_values(6,start_rowx=1,end_rowx=s.nrows)] #vector of correct actions
        BA = [int(num) for num in s.col_values(7,start_rowx=1,end_rowx=s.nrows)] #vector of best actions
            
        #update unavailable, correct and best action depending on horizontal order position AND on valuable token
        for tr in range(s.nrows-1):
            if to == 2:
                if tr_list_all['goalToken'][tp] == 1:
                    tr_list_all['goalToken'][tp] = 2
                elif tr_list_all['goalToken'][tp] == 2:
                    tr_list_all['goalToken'][tp] = 3
                elif tr_list_all['goalToken'][tp] == 3:
                    tr_list_all['goalToken'][tp] = 1
                    
            if (ho[run_nb-1] == 2 and to == 2) or (ho[run_nb-1] == 1 and to == 1):
                UAt = UA[tr]
                CAt = CA[tr]
                BAt = BA[tr]     
            elif (ho[run_nb-1] == 1 and to == 2) or (ho[run_nb-1] == 3 and to == 1):      
                if UA[tr] == 1:
                    UAt = 2
                elif UA[tr] == 2:
                    UAt = 3
                elif UA[tr] == 3:
                    UAt = 1 
                if CA[tr] == 1:
                    CAt = 2
                elif CA[tr] == 2:
                    CAt = 3
                elif CA[tr] == 3:
                    CAt = 1
                if BA[tr] == 1:
                    BAt = 2
                elif BA[tr] == 2:
                    BAt = 3
                elif BA[tr] == 3:
                    BAt = 1
            elif (ho[run_nb-1] == 3 and to == 2) or (ho[run_nb-1] == 2 and to == 1):
                if UA[tr] == 1:
                    UAt = 3
                elif UA[tr] == 2:
                    UAt = 1
                elif UA[tr] == 3:
                    UAt = 2
                if CA[tr] == 1:
                    CAt = 3
                elif CA[tr] == 2:
                    CAt = 1
                elif CA[tr] == 3:
                    CAt = 2
                if BA[tr] == 1:
                    BAt = 3
                elif BA[tr] == 2:
                    BAt = 1
                elif BA[tr] == 3:
                    BAt = 2
                       
            tr_list_all['unavAct'].append(UAt)
            tr_list_all['corrAct'].append(CAt)
            tr_list_all['bestAct'].append(BAt)
            tp = tp + 1
        run_nb = run_nb + 1
            
    tr_list_df = pd.DataFrame.from_dict(tr_list_all)
    tr_list_df.to_csv('trial_list_v' + str(l) + '.csv', index=False)
    tr_list_df.head()

1
2
3
4
5
6
7
8
9
10


In [8]:
tr_list_df

,runNb,runID,trialNb,trType,goalToken,uncertainty,unavAct,corrAct,bestAct,vertOrd,horizOrd
0,1,1,1,1,2,2,2,3,3,1,3
1,1,1,2,1,2,2,3,2,3,1,3
2,1,1,3,2,2,2,1,3,3,1,3
3,1,1,4,1,2,2,3,2,3,1,3
4,1,1,5,1,3,2,3,1,1,1,3
5,1,1,6,2,3,2,1,3,1,1,3
6,1,1,7,1,3,2,2,1,1,1,3
7,1,1,8,1,3,2,3,1,1,1,3
8,1,1,9,1,2,2,1,3,3,1,3
9,1,1,10,2,2,2,2,3,3,1,3
